# Логистическая регрессия. Практика

В этом задании вам предлагается спрогнозировать, купит клиент велосипед или нет, обучив логистическую регрессию.

In [1]:
# подключим библиотеки
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [2]:
# считаем данные
data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/bike_buyers_clean.csv')

In [3]:
# выводим первые 5 строк датафрейма
data.head(5)

,ID,Marital Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Purchased Bike
0,12496,Married,Female,40000,1,Bachelors,Skilled Manual,Yes,0,0-1 Miles,Europe,42,No
1,24107,Married,Male,30000,3,Partial College,Clerical,Yes,1,0-1 Miles,Europe,43,No
2,14177,Married,Male,80000,5,Partial College,Professional,No,2,2-5 Miles,Europe,60,No
3,24381,Single,Male,70000,0,Bachelors,Professional,Yes,1,5-10 Miles,Pacific,41,Yes
4,25597,Single,Male,30000,0,Bachelors,Clerical,No,0,0-1 Miles,Europe,36,Yes


In [4]:
# смотрим размер датафрейма
data.shape

(1000, 13)

Выведите статистики по категориальным признакам, чтобы посмотреть, сколько категорий в каждом категориальном (нечисловом) признаке.

Для этого можно воспользоваться методом `describe` из библиотеки pandas со значением параметра `include = 'object'`.

**Вопрос:** в каком категориальном признаке встречаются три различных значения?

In [5]:
data.describe(include='object')

,Marital Status,Gender,Education,Occupation,Home Owner,Commute Distance,Region,Purchased Bike
count,1000,1000,1000,1000,1000,1000,1000,1000
unique,2,2,5,5,2,5,3,2
top,Married,Male,Bachelors,Professional,Yes,0-1 Miles,North America,No
freq,539,509,306,276,685,366,508,519


Закодируйте все категориальные столбцы с двумя категориями следующим образом:  
самая часто встречающаяся категория превращается в 1, другая в 0.

In [6]:
# ваш код здесь
data['Marital Status'] = (data['Marital Status'] == 'Married').astype(int)
data['Gender'] = (data['Gender'] == 'Male').astype(int)
data['Home Owner'] = (data['Home Owner'] == 'Yes').astype(int)
data['Purchased Bike'] = (data['Purchased Bike'] == 'No').astype(int)

Удалите остальные категориальные столбцы.

**Вопрос:** сколько категориальных столбцов вы удалили?

In [7]:
# ваш код здесь
del data['Education']
del data['Occupation']
del data['Commute Distance']
del data['Region']

Удалите столбец `ID`, так как он по сути нечисловой.

In [8]:
# ваш код здесь
del data['ID']

Сформируйте матрицу объект-признак `X` и вектор `y` с целевой переменной.  
Целевая переменная - это последний столбец, `Purchased Bike`.

In [9]:
# ваш код здесь
y = data['Purchased Bike']
X = data.drop('Purchased Bike', axis=1)

Разбейте данные на тренировочную и тестовую часть (`Xtrain`, `Xtest`, `ytrain`, `ytest`), в тест отправьте 30% данных.  
Зафиксируйте `random_state = 42`.

In [10]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=42)

**Вопрос:** сколько объектов в матрице `Xtrain`?

In [11]:
# ваш код здесь
Xtrain.shape[0]

700

Почти всё готово для обучения модели!

Осталось отмасштабировать матрицу `X`, так как линейные модели чувствительны к масштабу данных.

*  Обучите на тренировочной матрице (`Xtrain`) `MinMaxScaler` из библиотеки `sklearn.preprocessing`
*  Примените масштабирование и к `Xtrain`, и к `Xtest`
*  Переведите полученные после масштабирования `np.array` обратно в pandas `dataframe`.

Полученные масштабированные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [12]:
from sklearn.preprocessing import MinMaxScaler

# ваш код здесь
scaler = MinMaxScaler()
Xtrain = pd.DataFrame(scaler.fit_transform(Xtrain), columns=Xtrain.columns, index=Xtrain.index)
Xtest = pd.DataFrame(scaler.transform(Xtest), columns=Xtest.columns, index=Xtest.index)

Теперь обучите логистическую регрессию на тренировочных данных

In [13]:
from sklearn.linear_model import LogisticRegression

# ваш код здесь
model = LogisticRegression()
model.fit(Xtrain, ytrain)

LogisticRegression()

Сделайте предсказания на тренировочных и на тестовых данных.

In [14]:
# ваш код здесь
prediction_train = model.predict(Xtrain)

prediction_test = model.predict(Xtest)

Оцените значение accuracy на трейне и на тесте.

In [15]:
from sklearn.metrics import accuracy_score

# ваш код здесь
accuracy_train = accuracy_score(ytrain, prediction_train)

accuracy_test = accuracy_score(ytest, prediction_test)

print(f"Accuracy on train = {round(accuracy_train,3)} \nAccuracy on test = {round(accuracy_test, 3)}")

Accuracy on train = 0.634 
Accuracy on test = 0.58


Качество модели получилось невысоким, зато модель не переобучена.

Попробуем добавить новых признаков в модель, используя [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html).

Создайте полиномиальные признаки degree = 2.

Как обычно:
*  `fit` делайте на тренировочных данных
*  `transform` и на тренировочных, и на тестовых данных. Затем верните результат к формату pandas `dataframe`.

Полученные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

Добавим полиномиальные признаки второй степени:

$$1, x_1, x_2 \to 1, x_1, x_2, x_1^2, x_2^2, x_1x_2.$$

In [16]:
# количество признаков в исходном датафрейме
Xtrain.shape[1]

7

In [17]:
from sklearn.preprocessing import PolynomialFeatures

# ваш код здесь
pf = PolynomialFeatures(degree = 2)

pf.fit(Xtrain)

Xtrain = pd.DataFrame(pf.transform(Xtrain), index=Xtrain.index)

Xtest = pd.DataFrame(pf.transform(Xtest), index=Xtest.index)

In [18]:
Xtrain.shape[1]

36

**Вопрос:** на сколько признаков стало больше при добавлении полиномиальных признаков второй степени?

Заново обучите логистическую регрессию, уже на расширенной матрице признаков, и сделайте предсказания на трейне и тесте, а затем оцените качество (*accuracy*).

In [19]:
# ваш код здесь
model2 = LogisticRegression()
model2.fit(Xtrain, ytrain)

prediction2_train = model2.predict(Xtrain)
prediction2_test = model2.predict(Xtest)

accuracy2_train = accuracy_score(ytrain, prediction2_train)
accuracy2_test = accuracy_score(ytest, prediction2_test)
print(f"Accuracy on train = {round(accuracy2_train,3)} \nAccuracy on test = {round(accuracy2_test, 3)}")

Accuracy on train = 0.687 
Accuracy on test = 0.623


**Вопрос:** на сколько повысилось качество модели на тестовых данных?  
Ответ округлите до сотых.

In [20]:
round(accuracy2_test - accuracy_test, 2)

0.04

Появились новые требования от заказчика!

Заказчик просит, чтобы:
*  доля найденных моделью потенциальных покупателей была максимальной
*  accuracy при этом была не ниже, чем 0.6 (отклонения *accuracy* на тестовых данных на $\pm 0.05$ допустимы).

Сначала посмотрите, какие значения *recall* и *accuracy* имеют предсказния модели на тесте с классами, предсказанными по умолчанию (методом `predict`).

In [21]:
# ваш код здесь
from sklearn.metrics import recall_score

recall_2 = recall_score(ytest, prediction2_test)
print(f"Recall = {round(recall_2,3)} \nAccuracy = {round(accuracy2_test, 3)}")

Recall = 0.696 
Accuracy = 0.623


Подберём порог для перевода вероятностей в классы, чтобы оптимизировать требуемые метрики!

Разобъем тренировочные данные на трейн и валидацию, чтобы по валидационной части подбирать порог.

In [22]:
XtrainS, Xval, ytrainS, yval = train_test_split(Xtrain, ytrain, test_size=0.3, random_state=42)

XtrainS.shape, Xval.shape

((490, 36), (210, 36))

* Обучите модель на тренировочных данных.
* Предскажите вероятности положительного класса на валидационных данных

В цикле для каждого значения порога:
*  переведите вероятности в классы
*  вычислите полноту (на валидационных данных)

Выведите на экран:

1) значение порога, дающее максимальный *recall*, при условии, что *accuracy* $\geq$ 0.6.

2) значение *recall* при этом пороге

3) значение *accuracy* для этого порога


Ищите порог на отрезке от 0 до 1 с шагом 0.01.

In [23]:
from sklearn.metrics import recall_score

RecMax = -1
BestThr = -1
BestAcc = -1

model3 = LogisticRegression()
model3.fit(XtrainS, ytrainS)
predict_pos = model3.predict_proba(Xval)[:, 1]

for thr in np.arange(0, 1, 0.01):
    # ваш код здесь
    prediction3 = (predict_pos > thr).astype(int)
    recall_cur = recall_score(yval, prediction3)
    acc_cur = accuracy_score(yval, prediction3)
    if recall_cur >= RecMax and acc_cur >= 0.6:
        BestThr = thr
        RecMax = recall_cur
        BestAcc = acc_cur

print (BestThr, RecMax, BestAcc)

0.42 0.8518518518518519 0.6142857142857143


Теперь заново обучите модель на исходных тренировочных данных (`Xtrain`, `ytrain`), предскажите вероятности на тесте и переведите их в классы по найденному порогу.

In [26]:
# ваш код здесь
model_tot = LogisticRegression()
model_tot.fit(Xtrain, ytrain)
predict_pr = model_tot.predict_proba(Xtest)[:, 1]
prediction_tot = (predict_pr > 0.42).astype(int)
recall_tot = recall_score(ytest, prediction_tot)
print(round(recall_tot, 1))

0.8


In [28]:
accuracy_tot = accuracy_score(ytest, prediction_tot)
print(round(accuracy_tot, 2))

0.57


**Вопрос:** какое значение *recall* получилось на тестовых данных после подбора порога?  
Ответ округлите до десятых.

При помощи подбора порога удалось сильно увеличить значение *recall*!  
Однако, как видно, на тесте не удалось сохранить условие $accuracy \geq 0.6$ (но в допустимые рамки уложились!)

Это свидетельство небольшого переобучения модели. Однако в сухом остатке имеет значительное увеличение полноты, что является приоритетом для заказчика.